In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [21]:
# y_t is either a vector of the solution equation evaluated at all points t in [0,T] OR
# a vector of the approximate solution values to y' = f(y;theta) at all points t in [0,T],
# as calculated by our time integrator below
# data is of form [(t,y_hat)], where y_hat is a data point collected at time t that should be compared against
# y_n[index][1] for each data point as y[n] is of form [(t,y_i)] where y_i is an approx. sol
def get_error(y_n,N,T,data):
    error = 0
    h = T/N # calculate timestep between each data point
    print(f"(N,T,h): ({N},{T},{h})")
    for index, (t,y_hat) in enumerate(data):
        print(f"data: ({y_hat})")
        print(f"approx (t,y_N):{y_n[N]}") # y_i
        error += (y_n[N*t] - y_hat) ** 2
    return error

In [22]:
#### NUMPY VERSION OF EULER'S
def forward_solve_euler_numpy(y_0,f,N,T,theta):
    sols = np.zeros(N+1)
    sols[0] = y_0
    h = T/N # timestep length

    # Calculate solutions
    for i in range(N):
        sols[i+1] = sols[i] + h * f(sols[i],theta)

    return sols

In [23]:
### Euler's Method
def forward_solve_euler_method(y_0,f,N,T,theta):
    y_ns = [y_0] # initialize with initial value y_0
    h = T/N # timestep length
    # Calculate solutions
    for i in range(N):
        y_i_plus_1 = y_ns[i] + h * f(y_ns[i],theta)
        y_ns.append(y_i_plus_1)
    return y_ns

In [24]:
# data is of form [(t,y_hat)]
# y_n is of form [(t,y_i)] where y_i is an approx. sol
# f_partial_y is the partial derivative of the function f w.r.t. y
def backward_solve_lagrange(data, y_ns, h, f_partial_y, theta):
    y_hat = data[0][1]
    y_N = y_ns[-1] # use last y_n as y_N to compute lambda_N
    lambda_N = 2 * (y_hat - y_N) ## Define lambda_N from error of (y_N - y^hat)^2
    lambda_ns = [lambda_N] ## Initialize lambda list
    y_ns_rev = y_ns[:-1][::-1] # Remove y_N; reverse y_ns to work backward
    for y_j in y_ns_rev:
        lambda_j = lambda_ns[0] # lambda_j is first element of list
        lambda_j_minus_1 = lambda_j + (lambda_j * h * f_partial_y(y_j,theta))
        lambda_ns = [lambda_j_minus_1] + lambda_ns # Add lambda_j_minus_1 to beginning of lambda_ns
    return lambda_ns

In [25]:
# data is of form [(t,y_j)] where y_j is an approx. sol
# lambda_ns is of form [lambda_j]; y_ns is of form [y_j]
# f_partial_theta is the partial derivative of the function f(y;theta) w.r.t. theta
def L_partial_theta(lambda_ns, h, T, N, f_partial_theta, y_ns, theta):
    grad = 0
    for j in range(1,N+1):
        grad += -1 * lambda_ns[j] * h * f_partial_theta(y_ns[j-1],theta)
    return grad

In [26]:
def gradient_descent(theta_init, M, alpha, data, y_0, f, f_partial_y, f_partial_theta, T, N):
    theta_i = theta_init
    gradient = 0
    for i in range(M):
        gradient = get_gradient(theta_i, data, y_0, f, f_partial_y, f_partial_theta, T, N)
        theta_i = theta_i + (-1 * alpha * gradient)
    return theta_i

In [27]:
# Generalized gradient descent; may require more parameters
def gradient_descent_general(theta_init,M,alpha,data,y_0,...):
    theta = theta_init
    error_partial_theta = 0
    for i in range(M): # iterate M times
        #get_gradient(...) is a n arbitrary function
        error_partial_theta = get_gradient(theta,data,y_0,...)
        theta = theta - (alpha * error_partial_theta)
    return theta

SyntaxError: invalid syntax (3114045382.py, line 2)

In [28]:
import math
# Gradient descent for y'= theta * y, data={(1,3)}
def gradient_descent_specific(theta_init,M,alpha):
    theta = theta_init
    error_partial_theta = 0 # initialize gradient variable
    for i in range(M): # iterate M times
        error_partial_theta = (2 * (math.exp(theta)-3) * math.exp(theta))
        theta = theta - alpha * error_partial_theta
    return theta
print(gradient_descent_specific(7,10000000,.000001))

1.0986122886742775


In [29]:
def get_gradient(theta, data, y_0, f, f_partial_y, f_partial_theta, T, N):
    y_ns = forward_solve_euler_method(y_0,f,N,T,theta) ## Forward solve for y_ns
    #print(len(y_ns))
    #print('y_ns',y_ns)
    lambda_ns = backward_solve_lagrange(data, y_ns, h, f_partial_y, theta) ## Backward solve for lambda_ns
    #print('lambda_ns',lambda_ns)
    grad = L_partial_theta(lambda_ns, h, T, N, f_partial_theta, y_ns, theta) ## gradient with y_ns, lambda_ns
    return grad

In [30]:
def euler_formula(y_0,f,N,T,theta,n):
    return (1 + (theta)/N) ** n

In [31]:
def compute_gradient_formula():
    pass

In [32]:
def f_of_y_ex(theta,y):
    return theta * y

In [33]:
def f_partial_y_ex(y,theta):
    return theta

In [34]:
def f_partial_theta_ex(y,theta):
    return y

In [35]:
#### SET UP ODE
theta_init = 1 # single parameter; should be generalized elsewhere
f = f_of_y_ex
y_prime = f
y_0 = 1 ## data point @ t=0, or y(0)

#### SET UP DATA
data = [(1,3)]

'''
#### SET UP ACTUAL SOLUTION y(t)
c = 1.5
y = lambda y: c*y
y_t = [y(1),y(2),y(3)]
'''

#### SET UP EULER TO OBTAIN APPROX SOL
N = 100 ## Number of timesteps
T = 1 ## set solution time interval as [0,T]
h = T/N ## timestep length
y_n = forward_solve_euler_method(y_0,f,N,T,theta_init) ## Get approx solution values at each time t in [0,T], where timestep length h is taken between each t

euler_y_n = euler_formula(y_0,f,N,T,theta_init,N)
#print('y_n approx of y:',y_n)

len(y_n)

error = get_error(y_n,N,T,data)
print('error',error)

(N,T,h): (100,1,0.01)
data: (3)
approx (t,y_N):2.704813829421526
error 0.08713487530078405


In [36]:
lag_grad = get_gradient(theta_init, data, y_0, f, f_partial_y_ex, f_partial_theta_ex, T, N)
print('lag_grad',lag_grad)

lag_grad -1.581036903830964


In [37]:
# theta_init defined
M = 10000 # Number of iterations of gradient descent
alpha = .01 # learning constant
# data defined above
# y_0 defined above
f = f_of_y_ex # f defined above but just redefining for completeness's sake
f_partial_y = f_partial_y_ex
f_partial_theta = f_partial_theta_ex
# T defined above
# N defined above
optimal_theta = gradient_descent(theta_init, M, alpha, data, y_0, f, f_partial_y, f_partial_theta, T, N)
print(optimal_theta)

1.1046691937853583
